In [1]:
!pip install langchain beautifulsoup4 langchain_community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 18.8 MB/s eta 0:0

In [2]:
from google.colab import userdata
import huggingface_hub

hf_token = userdata.get('HF_TOKEN')

huggingface_hub.login(hf_token)

In [3]:
!mkdir web_pages

In [4]:
import os
import re
import requests
from bs4 import BeautifulSoup

urls = [
    "https://canonical.com/solutions/ai",
    "https://canonical.com/data"
]

headers = {
    "Accept": "application/json",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

def sanitize_filename(name: str) -> str:
    """Remove invalid characters and format for saving."""
    name = re.sub(r"[\\/*?\"<>|:]", "", name)
    name = name.replace(" ", "_")
    return name.strip() or "untitled_page"

root_data_path = "web_pages"

for url in urls:
    print(f"Fetching {url}...")
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    # Get title
    title_tag = soup.find("title")
    title = title_tag.get_text(strip=True) if title_tag else "Untitled Page"
    safe_title = sanitize_filename(title)
    filename = f"{root_data_path}/{safe_title}.txt"

    # Find main content
    soup_main = soup.find("div", id="main-content")

    with open(filename, "w", encoding="utf-8") as f:
        f.write(f"URL: {url}\n")
        f.write(f"Title: {title}\n\n")

        if not soup_main:
            f.write("No main-content div found.\n")
            print(f"⚠️  No main-content found for {url}")
            continue

        sections = soup_main.find_all("section", class_="p-section")
        if not sections:
            f.write("No p-section sections found.\n")
            print(f"⚠️  No p-section found for {url}")
            continue

        # Write all text sections
        for idx, section in enumerate(sections, start=1):
            text = section.get_text(" ", strip=True)
            f.write(f"\n{text}\n\n")

    print(f"✅ Saved: {filename}")

Fetching https://canonical.com/solutions/ai...
✅ Saved: web_pages/Open_source_AI_for_the_enterprise.txt
Fetching https://canonical.com/data...
✅ Saved: web_pages/Data_Solutions.txt


In [5]:
import os
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

DATA_DIR = "./web_pages"
CHROMA_DIR = "./chroma_store"

# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Collect all text files
documents = []

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)
chunk_count = 0

print(f"Loading documents from {DATA_DIR}...")

for file_name in os.listdir(DATA_DIR):
    if file_name.endswith(".txt"):
        file_path = os.path.join(DATA_DIR, file_name)

        try:
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read().strip()

            if content:
                # Split the content into chunks
                chunks = text_splitter.split_text(content)

                # Create Document objects with metadata
                for i, chunk in enumerate(chunks):
                    doc = Document(
                        page_content=chunk,
                        metadata={
                            "source": file_name,
                            "chunk_id": i,
                            "total_chunks": len(chunks)
                        }
                    )
                    documents.append(doc)
                    chunk_count += 1

                print(f"{file_name}: {len(chunks)} chunks")

        except Exception as e:
            print(f"Error loading {file_name}: {e}")

# Create or load Chroma index
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory=CHROMA_DIR
)

vectorstore.persist()
print(f"✅ Chroma vector store created and persisted at: {CHROMA_DIR}")

/tmp/ipython-input-895305984.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loading documents from ./web_pages...
Open_source_AI_for_the_enterprise.txt: 14 chunks
Data_Solutions.txt: 5 chunks
✅ Chroma vector store created and persisted at: ./chroma_store


/tmp/ipython-input-895305984.py:63: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [6]:
import os
import torch
import requests
import transformers
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# ========================
# CONFIGURATION
# ========================
CHROMA_DIR = "./chroma_store"
TOMORROW_API_KEY = "GPX9zR2cVa94DjRHnpcKyQIn0nYxTGMd"
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"

# ========================
# LOAD VECTOR DB
# ========================
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = Chroma(
    persist_directory=CHROMA_DIR,
    embedding_function=embedding_model
)

# ========================
# MODEL + TOKENIZER
# ========================
model_id = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

/tmp/ipython-input-1169165928.py:20: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Device set to use cuda:0


In [54]:
from typing import Optional
import re

def get_weather(city: str):
    """Fetch live weather information using Tomorrow.io Realtime API."""
    try:
        url = f"https://api.tomorrow.io/v4/weather/realtime?location={city}&apikey={TOMORROW_API_KEY}"

        headers = {
            "accept": "application/json",
            "accept-encoding": "deflate, gzip, br"
        }

        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        data = response.json()
        values = data["data"]["values"]
        temp = values.get("temperature")
        weather_code = values.get("weatherCode")

        # Map weather code to description
        description = get_weather_description(weather_code) if weather_code else "unknown conditions"

        print(f"Weather fetched for {city}: {temp}°C")
        return f"The weather in {city} is currently {description} with a temperature of {temp}°C."

    except requests.RequestException as e:
        return f"Error fetching weather data: {e}"
    except KeyError as e:
        return f"Error: unexpected response format from weather API. Missing: {e}"
    except Exception as e:
        return f"Unexpected error: {e}"


def get_weather_description(code: int) -> str:
    """Map Tomorrow.io weather codes to human-readable descriptions."""
    weather_codes = {
        0: "Unknown",
        1000: "Clear, Sunny",
        1100: "Mostly Clear",
        1101: "Partly Cloudy",
        1102: "Mostly Cloudy",
        1001: "Cloudy",
        2000: "Fog",
        2100: "Light Fog",
        4000: "Drizzle",
        4001: "Rain",
        4200: "Light Rain",
        4201: "Heavy Rain",
        5000: "Snow",
        5001: "Flurries",
        5100: "Light Snow",
        5101: "Heavy Snow",
        6000: "Freezing Drizzle",
        6001: "Freezing Rain",
        6200: "Light Freezing Rain",
        6201: "Heavy Freezing Rain",
        7000: "Ice Pellets",
        7101: "Heavy Ice Pellets",
        7102: "Light Ice Pellets",
        8000: "Thunderstorm"
    }
    return weather_codes.get(code, f"Weather code {code}")

def tool_selector(user_query: str) -> tuple[str, str]:
    """
    Uses the LLM to classify the user's query into one of three categories (general, weather, both)
    Returns: (tool_name, reasoning)
    """
    classification_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Classify the query into **exactly** one of the following categories:
- weather: when asked about anything related to weather
- general: when asked about anything but the weather
- general+weather: when asked about weather and other information

Respond in this **exact** format:
Category: [category_name]
Reason: [one short sentence explaining why]<|eot_id|><|start_header_id|>user<|end_header_id|>

Query: What is Canonical?
Category:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Category: general
Reason: The query asks about a company/product, not weather.<|eot_id|><|start_header_id|>user<|end_header_id|>

Query: What's the weather in London?
Category:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Category: weather
Reason: The query explicitly asks about weather conditions.<|eot_id|><|start_header_id|>user<|end_header_id|>

Query: What is Canonical and how is the weather in London?
Category:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Category: general+weather
Reason: The query asks about both company information and weather.<|eot_id|><|start_header_id|>user<|end_header_id|>

Query: {user_query}
Category:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

    output = pipeline(
        classification_prompt,
        max_new_tokens=50,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
    )

    full_output = output[0]["generated_text"][len(classification_prompt):].strip()

    category = "general"
    reason = "No classification found"

    # Extract category
    category_match = re.search(r"Category:\s*(general\+weather|weather|general)", full_output, re.IGNORECASE)
    if category_match:
        category = category_match.group(1).lower()
    else:
        full_output_lower = full_output.lower()
        if "general+weather" in full_output_lower or "general and weather" in full_output_lower:
            category = "general+weather"
        elif "weather" in full_output_lower:
            category = "weather"
        elif "general" in full_output_lower:
            category = "general"

    # Extract reason
    reason_match = re.search(r"Reason:\s*(.+?)(?:\n|<|$)", full_output, re.IGNORECASE)
    if reason_match:
        reason = reason_match.group(1).strip()

    return category, reason


def agent_respond(user_query: str, vectorstore: Optional[object] = None) -> str:
    """
    ReAct-style agent that:
      1. Uses LLM-based `tool_selector` to decide which tools to use.
      2. Collects only the required context.
      3. Generates the final answer using the chosen context(s).
    """
    print("🔍 Selecting tools...")
    selected_tool, reasoning = tool_selector(user_query)
    print(f"Tool selected: {selected_tool}")

    context = ""
    weather_info = ""

    # Retrieve from vectorstore if "general" or "general+weather"
    if selected_tool in ["general", "general+weather"] and vectorstore:
        try:
            results = vectorstore.similarity_search(user_query, k=5)
            if results:
                print("🧠 Getting data from knowledge base")
                context += "\n\n--- Retrieved Knowledge ---\n"
                for doc in results:
                    context += f"{doc.page_content}\n"
        except Exception as e:
            print(f"⚠️ Vector store error: {e}")

    # Get weather info if "weather" or "general+weather"
    if selected_tool in ["weather", "general+weather"]:
        detected_city = None
        cities = ["Bangalore", "Delhi", "London", "New York", "Tokyo", "Toronto", "Mumbai", "Paris"]

        for city in cities:
            if city.lower() in user_query.lower():
                detected_city = city
                break

        if detected_city:
            print(f"🌦️ Fetching weather for {detected_city}...")
            weather_info = get_weather(detected_city)
        else:
            weather_info = "Weather was requested, but no specific city name was detected."

    # System prompt
    system_prompt = """You are a helpful assistant. Follow these rules:
    1. Judge from the prompt if question can be answered with information from knowledge base or weather information or both. Below are some examples for your reference.
    Example:
    Context: "Canonical offers data solutions including PostgreSQL, MySQL, and MongoDB."
    Question: "What databases does Canonical support and how is the weather in london?"
    Answer: "Based on the information provided, Canonical offers support for several databases including PostgreSQL, MySQL, and MongoDB as part of their data solutions. The Weather fetched for Delhi is 28.6°C"

    Now answer the user's question using ONLY the context provided below. If no relevant context exists, say so."""

    # Include only the contexts relevant to the chosen tool(s)
    user_content = f"""
    Question: {user_query}

    Below information contains retrieved knowledge and weather information (based on selected tools):

    {f"Context from knowledge base:\n{context}" if context and selected_tool in ['general', 'general+weather'] else ""}
    {f"Current weather information:\n{weather_info}" if weather_info and selected_tool in ['weather', 'general+weather'] else ""}
    """

    # Compose full prompt
    prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{user_content}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

    print("🧩 Generating final response...")
    output = pipeline(
        prompt,
        max_new_tokens=300,
        do_sample=False,
        temperature=None,
        top_p=None,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )

    # Extract generated text
    full_output = output[0]["generated_text"]
    response_text = full_output[len(prompt):].strip()
    response_text = response_text.split("<|eot_id|>")[0].strip()
    response_text = response_text.split("<|end_of_text|>")[0].strip()

    # Add reasoning note
    reasoning_note = f"\n\n[Agent Thought: The LLM decided to use '{selected_tool}' tool(s) because: {reasoning}]"
    return response_text + reasoning_note

In [ ]:
if __name__ == "__main__":
    print("\n" + "=" * 60)
    print("🤖 LLAMA-3.2 AGENT WITH WEATHER INTEGRATION")
    print("=" * 60)
    while True:
      user_query = input("You: ").strip()
      if user_query.lower() in ["exit", "quit"]:
          print("👋 Exiting chat. Goodbye!")
          break
      response = agent_respond(user_query, vectorstore)
      print(f"\n🧠 Agent response: {response}")


🤖 LLAMA-3.2 AGENT WITH WEATHER INTEGRATION
You: tell me a little about canonical
context >>>>>> 

--- Retrieved Knowledge ---
URL: https://canonical.com/data
Title: Data Solutions
Use cases Canonical offers you the building blocks so you can innovate at your own pace.
          From getting started with data science on Ubuntu workstations to scaling your big data analytics with supported database and MLOps software. All on open source. Confidential AI Data science AI infrastructure Data management MLOps GenAI with RAG Big data analytics Edge AI
URL: https://canonical.com/solutions/ai
Title: Open source AI for the enterprise


Why Canonical for enterprise AI? Run your entire AI/ML lifecycle on a single integrated stack Develop at all scales with the same software provider Control your TCO with predictable costs Get maintained and supported open source AI software
Designed for any silicon Improve the performance of AI workflows and accelerate project delivery by taking full advantage of

KeyboardInterrupt: Interrupted by user

In [55]:
agent_respond("tell me abut canonical offerings", vectorstore)

🔍 Selecting tools...
Tool selected: general
🧠 Getting data from knowledge base
🧩 Generating final response...


'Based on the provided context, Canonical offers various data solutions including:\n\n* PostgreSQL\n* MySQL\n* MongoDB\n\nThese databases are mentioned under the section "Learn more about [database name" in the context.\n\n[Agent Thought: The LLM decided to use \'general\' tool(s) because: The query asks about company offerings, not weather.]'

In [56]:
agent_respond("How is the weather in delhi right now?", vectorstore)

🔍 Selecting tools...
Tool selected: weather
🌦️ Fetching weather for Delhi...
Weather fetched for Delhi: 29.8°C
🧩 Generating final response...


"The current weather in Delhi is Clear, Sunny with a temperature of 29.8°C.\n\n[Agent Thought: The LLM decided to use 'weather' tool(s) because: The query asks about current weather conditions.]"

In [57]:
agent_respond("how can i use canonical for enterprise and how is the weather in delhi right now?", vectorstore)

🔍 Selecting tools...
Tool selected: general+weather
🧠 Getting data from knowledge base
🌦️ Fetching weather for Delhi...
Weather fetched for Delhi: 29.8°C
🧩 Generating final response...


'Based on the provided context, I can answer the question "how can I use Canonical for enterprise" as follows:\n\nCanonical offers various solutions for enterprises, including:\n\n* Building blocks for innovation through open-source software\n* A single integrated stack for running the entire AI/ML lifecycle\n* Predictable costs and control over Total Cost of Ownership (TCO)\n* Professional services to help bridge the skills gap and overcome barriers to AI adoption\n* Data solutions that simplify operations at any scale, including advanced automation for open-source data systems.\n\nHowever, I couldn\'t find any direct information on how to use Canonical specifically for enterprise AI/ML development. It seems that the provided context focuses more on the benefits and services offered by Canonical rather than providing detailed instructions on how to implement them.\n\nAs for the second part of the question, "and how is the weather in Delhi right now?", according to the current weather 